# Prototype Rich

In [1]:
from rich import print as pr
from fastcore.all import L, patch
from fastcore.xtras import sparkline
from rich.columns import Columns
from rich.console import RenderGroup
from rich.table import Table
from rich.padding import Padding
from rich.console import Console
from rich.progress import Progress, Spinner, BarColumn
from rich.live import Live
from random import randint
from collections import deque
from rich import spinner
from rich.panel import Panel
import time
from fastcore.xtras import sparkline
data = [9,6,None,1,4,1,8,15,10]

In [2]:
h_cmt = deque([1], 5)
h_iss = deque([1], 5)
h_pr = deque([1], 5)
h_rel = deque([1], 5)

In [3]:
def update_stats(d:dict=dict(c_cmt=0, c_iss=0, c_pr=0, c_rel=0)):
    
    h_cmt.append(d['c_cmt'])
    h_iss.append(d['c_iss'])
    h_pr.append(d['c_pr'])
    h_rel.append(d['c_rel'])
    
    sd = dict(s_cmt=sparkline(h_cmt), s_iss=sparkline(h_iss), s_pr=sparkline(h_pr), s_rel=sparkline(h_rel))
    d.update(sd)
    
    row = (':heavy_plus_sign: Commits  \n[blue]{c_cmt} {s_cmt}  [/] ',
            ':ticket: Issues  \n[red]{c_iss} {s_iss} [/]',
            ':handshake: PRs  \n[deep_pink1]{c_pr} {s_pr} [/]',
            ':rocket: Releases  \n[green]{c_rel} {s_rel} [/]')

    grid = Table.grid()
    for c in ['Commits','Issues', 'PRs', 'Releases']:
        grid.add_column(c, justify='center')
    grid.add_row(*(x.format(**d) for x in row))
    
    return grid

def _r(): return randint(10,100)


In [7]:
quotap = Progress(
    "[progress.description]{task.description}",
    BarColumn(bar_width=5),
    "[progress.percentage]{task.percentage:>3.0f}%",
)
qtask = quotap.add_task("", total=100)
qgrid = Table.grid()
qgrid.add_column(justify='center')
qgrid.add_row("Quota")
qgrid.add_row(quotap.get_renderable())


counts = update_stats()
qgroup = RenderGroup("Quota", quotap.get_renderable())

stat_group = Columns([RenderGroup(Padding(Spinner('earth'), [0,0,0,4]), 'events/s'), counts, qgrid], 
                      column_first=True, align='center')

Output()

In [9]:
with Live(refresh_per_second=5) as live:
    for _ in range(100):
        quotap.update(qtask, advance=1)
#         stat_group.renderables[2] = quotap.get_renderable()
        stat_group.renderables[2].columns[0]._cells[1] = quotap.get_renderable()
        d=dict(c_cmt=_r(), c_iss=_r(), c_pr=_r(), c_rel=_r())
        new_count = update_stats(d)
        stat_group.renderables[1] = new_count
        live.update(stat_group)
        time.sleep(0.1)

Output()

In [6]:
from fastcore.xtras import sparkline
data = [9,6,None,1,4,1,8,15,10]
print(f'without "empty_zero": {sparkline(data, empty_zero=False)}')
print(f'   with "empty_zero": {sparkline(data, empty_zero=True )}')

without "empty_zero": ▃▂ ▁▁▁▃▇▅
   with "empty_zero": ▃▂ ▁▁▁▃▇▅


TODO:

- Use emojis from `ghapi.event`
- Refactor this code so its not so ugly and repetitive